# Physics 210 Project 1: "How can colliding with the slime planet save the spaceship from a black hole?"

**Name:** Mufaro Joseph Machaya

## Introduction

**Research Question:** Given a three-body system containing a black hole (with a mass $M$), a small planet made of JELL-O-like slime (with mass $m_p$), and a spaceship (with mass $m_s$) accidentally headed directly for the black hole, the only way for the spaceship to avoid being sucked into the black hole is to turn into and collide (inelastically) with the planet such that it is successfully rebounded with a velocity high enough to escape the gravitational pull of the black hole. However, if it hits the planet with too much impulse during the collision, it'll be destroyed by the planet (it's made out of JELL-O, so it has surface tension). 

*With what range of initial velocities and what turn radius/time allow the ship to successfully avoid being sucked into the black hole without being destroyed by the planet?*

### Physics Model and Assumptions

**Model**

1. $n$-Body System, Gravitational Vector Field

Each of the three objects are modeled individually as point-masses. Each object produces gravitational effects on all other objects by formula

$$\vec{F}_g = G \frac{m_1 m_2}{|\vec{r}|^2} \hat{r}.$$

From there, basic kinematic equations apply:

$$\frac{d^2 \vec{x}}{dt^2} = \frac{\sum F}{m}.$$

The system is modeled in only two dimensions rather than three (but all equations apply for each dimension)

2. Spaceship Turning

The ship will need to perform a turn of angle $\theta$ to have a proper head-on collision. This will require the ship to apply thrust to perform the turn, but this turn will require a certain amount of energy. Let's assume that the ship starts with a certain amount of fuel on board (but it will require fuel to accelerate/deccelerate during its journey).

3. Inelastic Jello Collision

The force from interacting with a gel-like substance will be modeled using a hunt-crossley algorithm with optional adhesion and a plastic offset, i.e.,

- Contact when the overlap distance $\delta > 0$, and the force for each frame is $$F_N = k \delta^n + \alpha \delta^n \frac{d \delta}{dt},$$ with $n=\frac{3}{2}$ for spherical geometry (and $\delta = R_1 + R_2 - |\vec{r}_1 - \vec{r}_2|$)

**Assumptions**

1. $m_s < m_p <<< M$: The mass of the black hole is so much larger than the mass of the planet and ship that the black hole is taken to be stationary.
2. This model assumes no relativistic effects whatsoever (due to all speeds $v$ being much lower than the speed of light $c$.
3. The ship is presumed to instantaneously make a full $180^\circ$ turn immediately after its collision.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from dataclasses import dataclass

In [ ]:
@dataclass
class RigidBody:
    mass: float
    color: str
    name: str

    stationary: bool = False 
    track:      bool = False
    
    initial_position:     float = 0
    initial_velocity:     float = 0
    initial_acceleration: float = 0
    
    time:         np.array = None
    position:     np.array = None
    velocity:     np.array = None
    acceleration: np.array = None
    energy:       np.array = None
    momentum:     np.array = None
    
    def init(self, n):
        self.position     = np.zeros(n)
        self.velocity     = np.zeros(n)
        self.acceleration = np.zeros(n)
        self.energy       = np.zeros(n)
        self.momentum     = np.zeros(n)
        
        self.position[0]     = self.initial_position
        self.velocity[0]     = self.initial_velocity
        self.acceleration[0] = self.initial_acceleration
        
    def __hash__(self):
        return hash(self.name)
        
    def distance_to(self, other_body, i):
        return np.sqrt(np.sum((self.position[i] - other_body.position[i])**2))

    def collided_with(self, other_body, collision_radius, i):
        return self.distance_to(other_body, i) <= collision_radius
    
    def near_barrier(self, bottom_barrier, top_barrier, collision_radius, i):
        return \
            abs(self.position[i] - bottom_barrier) <= collision_radius or \
            abs(self.position[i] - top_barrier) <= collision_radius

In [ ]:
def apply_gravitational_field(i, body, objects):
    if body.stationary:
        return
    
    other_objects = [x for x in objects if x is not body]

In [ ]:
def simulate(objects, time_range, dt, 
             collision_radius=0.1, 
             bottom_barrier=-10, 
             top_barrier=10):
    time = np.arange(time_range[0], time_range[1], dt)
    total_frames = np.size(time)
    collisions = set()
    
    for body in objects:
        body.init(total_frames)
    
    for i in range(1, total_frames):
        # update positions
        for body in objects:
            if body.stationary = True:
                continue
            
            body.position[i] = body.position[i-1] + body.velocity[i-1]     * dt
            body.velocity[i] = body.velocity[i-1] + body.acceleration[i-1] * dt
            body.acceleration[i] = apply_gravitational_field(i, body, objects)
        
        # detect collisions
        to_invert = set()
        
        for first_body_idx, body in enumerate(objects):
            for other_body in objects[first_body_idx+1:]:
                if body.collided_with(other_body, collision_radius, i):
                    collisions.add((time[i], body.position[i]))
                    to_invert.add(body)
                    to_invert.add(other_body)
                    
        for body in objects:
            if body.near_barrier(bottom_barrier, top_barrier, collision_radius, i):
                to_invert.add(body)
        
        for body in to_invert:
            body.velocity[i] *= -1
            
    # ending calculations
    for body in objects:
        body.momentum = body.mass * abs(body.velocity)
        body.energy = (1/2) * body.mass * body.velocity ** 2
        
    # transpose collisions
    collisions = np.array(list(zip(*collisions)))
        
    return time, collisions

In [ ]:
def plot_results(objects, time, collisions):
    fig, axs = plt.subplots(4,1, sharex=True, figsize=(8,8))

    # position
    for body in objects:
        axs[0].plot(time, body.position, 
                    color=body.color, 
                    label=body.name,
                    linestyle='dashed')
    axs[0].scatter(collisions[0], collisions[1], label='Collisions')
    axs[0].axhline(y=10, color='black', label='Top Barrier')
    axs[0].axhline(y=-10, color='black', label='Bottom Barrier')
    axs[0].set_ylabel('Position\n(m)')
    axs[0].set_title('Position vs. Time')
    axs[0].legend()

    # velocity
    for body in objects:
        axs[1].plot(time, body.velocity, 
                    color=body.color, 
                    label=body.name)
    axs[1].set_ylabel('Velocity\n(m/s)')
    axs[1].set_title('Velocity vs. Time')
    axs[1].legend()

    # kinetic energy
    total_energy = np.zeros_like(time)
    for body in objects:
        axs[2].plot(time, body.energy * 1e3, 
                    color=body.color, 
                    label=body.name)
        total_energy = total_energy + body.energy
    axs[2].plot(time, total_energy * 1e3, color='green', label='Total Kinetic Energy')
    axs[2].set_ylabel('Kinetic Energy\n(mJ)')
    axs[2].set_title('Kinetic Energy vs. Time')
    axs[2].legend()

    # momentum
    total_momentum = np.zeros_like(time)
    for body in objects:
        axs[3].plot(time, body.momentum * 1e2, 
                    color=body.color, 
                    label=body.name)
        total_momentum = total_momentum + body.momentum
    axs[3].plot(time, total_momentum * 1e2, color='green', label='Total Momentum')
    axs[3].set_xlabel('Time (s)')
    axs[3].set_ylabel('Momentum\n(kg*m/s*10^2)')
    axs[3].set_title('Momentum vs. Time')
    axs[3].legend()

    plt.show()

def simulate_and_plot(objects, time_range, dt):
    time, collisions = simulate(objects, time_range, dt)
    plot_results(objects, time, collisions)

In [ ]:
black_hole=RigidBody(
    mass  = 0.1,
    color = 'black',
    name  = 'Black Hole'
)

space_ship=RigidBody(
    mass  = 
    color = ''
    name  = ''
)

Given $\rho_{sl}$, the density of slime (just assumed based on some calculations I nabbed off Reddit) and $r_{\sl}$, the radius of the slime planet (which I'll probably approximate to be the size of our Moon), we can determine the mass of the planet for gravitational pull reasons.

In [ ]:

SLIME_DENSITY=1.048 # g/cm^3
SLIME_PLANET_RADIUS=1737.4 # km

slime_planet=RigidBody(
    mass  =  
)